In [1241]:
from IPython.core.display import HTML
HTML('''<script> </script> <form action="javascript:IPython.notebook.execute_cells_below()"><input type="submit" id="toggleButton" value="Refresh Me"></form>''')

In [1242]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [1243]:
import boto3
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.width', None)

"""
ec2 = boto3.client('ec2', region_name='us-west-2')
response = ec2.describe_instances()
print(response)oro
"""

def highlight_reserved(df_row):
    if df_row.name[0] in ['r4.8xlarge','m5.12xlarge']:
        return ['background-color: yellow; border: 1px  black solid !important; color: black !important']*2
    else:
        return ['border: 1px  black solid !important; color: black !important']*2
    
emr = boto3.client('emr' , region_name='us-west-2')

page_iterator = emr.get_paginator('list_clusters').paginate(
    ClusterStates=['RUNNING','WAITING']
)

clusters_active = []
for page in page_iterator:
    clusters_active.extend(page['Clusters'])

asg = boto3.client('autoscaling', region_name='us-west-2')
asg_details = asg.describe_auto_scaling_groups(AutoScalingGroupNames=['stx-usw2-ehc-presto-worker-asg'])
presto_cnt = int(len(asg_details['AutoScalingGroups'][0]['Instances']) + 2)

print("Total Active Clusters:", len(clusters_active) + 1)

emr_df = pd.DataFrame()
for cluster in clusters_active:
    cluster_id = cluster['Id']
    instance_groups = emr.list_instance_groups(ClusterId=cluster_id)['InstanceGroups']
    instance_groups_df = pd.DataFrame(instance_groups)
    instance_groups_df['Cluster_Name'] = cluster['Name']
    instance_groups_df['Cluster_Id'] = cluster['Id']
    emr_df = emr_df.append(instance_groups_df[['Cluster_Name','Cluster_Id','InstanceType', 'InstanceGroupType', 'Market', 'RunningInstanceCount' ]], ignore_index=True)

emr_df = emr_df.append({'Cluster_Name':'stx-usw2-ehc-presto-ec2', 'Cluster_Id':'' , 'InstanceType': 'r4.8xlarge', 'InstanceGroupType': '', 'Market':'ON_DEMAND', 'RunningInstanceCount': presto_cnt }, ignore_index=True)

instance_total = emr_df['RunningInstanceCount'].sum()
print('Total Running instances count within EMR: {}'.format(instance_total))
#instance_type_cnt = instance_type_grp['RunningInstanceCount'].agg({'Count': np.sum})
#instance_type_cnt = instance_type_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Instance' : lambda x: ', '.join(x)})
instance_grp = emr_df.groupby(['InstanceType','Market'])
#instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum'})
instance_grp_cnt = instance_grp.agg({'RunningInstanceCount' : 'sum', 'Cluster_Name' : lambda x: ', '.join(x)}).reindex(['RunningInstanceCount','Cluster_Name'], axis=1)
instance_grp_cnt = instance_grp_cnt[instance_grp_cnt['RunningInstanceCount'] != 0]
print("Count of InstanceType accross EMR")
#instance_grp_cnt = instance_grp_cnt.style.apply(highlight_reserved, axis=1)
display(instance_grp_cnt)

instance_emr_cnt = emr_df.groupby(['InstanceType','Cluster_Name','Market']).agg({'RunningInstanceCount' : 'sum'})
instance_emr_cnt = instance_emr_cnt[instance_emr_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType per EMR")
display(instance_emr_cnt)

emr_instance_cnt = emr_df.groupby(['Cluster_Name','InstanceType','Market']).agg({'RunningInstanceCount' : 'sum'})
emr_instance_cnt = emr_instance_cnt[emr_instance_cnt['RunningInstanceCount'] != 0]
print("Distribution of InstanceType on EMR")
display(emr_instance_cnt)

print("Distribution of Instances per Cluster")
emr_node_per_cluster = emr_df.groupby(['Cluster_Name']).agg({'RunningInstanceCount' : 'sum'})
display(emr_node_per_cluster)


Total Active Clusters: 7
Total Running instances count within EMR: 62
Count of InstanceType accross EMR


RunningInstanceCount  \
InstanceType Market                            
m4.4xlarge   ON_DEMAND                     2   
m5.12xlarge  ON_DEMAND                     9   
m5.24xlarge  ON_DEMAND                     1   
r4.16xlarge  ON_DEMAND                     6   
r4.8xlarge   ON_DEMAND                    41   
             SPOT                          3   

                                                             Cluster_Name  
InstanceType Market                                                        
m4.4xlarge   ON_DEMAND  stx-usw2-ehc-wafer-EMR-5.30_New_Cluster, stx-u...  
m5.12xlarge  ON_DEMAND  stx-usw2-ehc-wafer-EMR-5.30_New_Cluster, stx-u...  
m5.24xlarge  ON_DEMAND                 stx-usw2-etl-ehc-spark-on-hive-prd  
r4.16xlarge  ON_DEMAND  stx-usw2-ehc-prd-etl-cluster, stx-usw2-ehc-prd...  
r4.8xlarge   ON_DEMAND  stx-usw2-ehc-prd-etl-cluster, stx-usw2-ehc-prd...  
             SPOT       stx-usw2-ehc-wafer-EMR-5.30_New_Cluster, stx-u...

Distribution of InstanceType per EMR


RunningInstanceCount
InstanceType Cluster_Name                            Market                         
m4.4xlarge   stx-usw2-ehc-prd-wafer-infosec          ON_DEMAND                     1
             stx-usw2-ehc-wafer-EMR-5.30_New_Cluster ON_DEMAND                     1
m5.12xlarge  stx-usw2-ehc-prd-etl-cluster            ON_DEMAND                     1
             stx-usw2-ehc-prd-sampled                ON_DEMAND                     2
             stx-usw2-ehc-prd-wafer-infosec          ON_DEMAND                     4
             stx-usw2-ehc-wafer-EMR-5.30_New_Cluster ON_DEMAND                     1
             stx-usw2-etl-ehc-spark-on-hive-prd      ON_DEMAND                     1
m5.24xlarge  stx-usw2-etl-ehc-spark-on-hive-prd      ON_DEMAND                     1
r4.16xlarge  stx-usw2-ehc-prd-etl-cluster            ON_DEMAND                     4
             stx-usw2-ehc-prd-sparkdev16-h2o         ON_DEMAND                     2
r4.8xlarge   stx-usw2-ehc-prd-etl-cluster            ON_DEMAND                     5
             stx-usw2-ehc-prd-sparkdev16-h2o         ON_DEMAND                     2
             stx-usw2-ehc-prd-wafer-infosec          ON_DEMAND                     1
                                                     SPOT                          2
             stx-usw2-ehc-presto-ec2                 ON_DEMAND                    32
             stx-usw2-ehc-wafer-EMR-5.30_New_Cluster SPOT                          1
             stx-usw2-etl-ehc-spark-on-hive-prd      ON_DEMAND                     1

Distribution of InstanceType on EMR


RunningInstanceCount
Cluster_Name                            InstanceType Market                         
stx-usw2-ehc-prd-etl-cluster            m5.12xlarge  ON_DEMAND                     1
                                        r4.16xlarge  ON_DEMAND                     4
                                        r4.8xlarge   ON_DEMAND                     5
stx-usw2-ehc-prd-sampled                m5.12xlarge  ON_DEMAND                     2
stx-usw2-ehc-prd-sparkdev16-h2o         r4.16xlarge  ON_DEMAND                     2
                                        r4.8xlarge   ON_DEMAND                     2
stx-usw2-ehc-prd-wafer-infosec          m4.4xlarge   ON_DEMAND                     1
                                        m5.12xlarge  ON_DEMAND                     4
                                        r4.8xlarge   ON_DEMAND                     1
                                                     SPOT                          2
stx-usw2-ehc-presto-ec2                 r4.8xlarge   ON_DEMAND                    32
stx-usw2-ehc-wafer-EMR-5.30_New_Cluster m4.4xlarge   ON_DEMAND                     1
                                        m5.12xlarge  ON_DEMAND                     1
                                        r4.8xlarge   SPOT                          1
stx-usw2-etl-ehc-spark-on-hive-prd      m5.12xlarge  ON_DEMAND                     1
                                        m5.24xlarge  ON_DEMAND                     1
                                        r4.8xlarge   ON_DEMAND                     1

Distribution of Instances per Cluster


,RunningInstanceCount
Cluster_Name,
stx-usw2-ehc-prd-etl-cluster,10
stx-usw2-ehc-prd-sampled,2
stx-usw2-ehc-prd-sparkdev16-h2o,4
stx-usw2-ehc-prd-wafer-infosec,8
stx-usw2-ehc-presto-ec2,32
stx-usw2-ehc-wafer-EMR-5.30_New_Cluster,3
stx-usw2-etl-ehc-spark-on-hive-prd,3
